#⚖️ Bootcamp: AI-Powered Virtual Referee Assistant (VAR/Umpire)

##🔍 Use Case Examples:
- **Cricket**: Detect LBW, no-balls, swing angle
- **Football (Soccer)**: Detect offside positions or handballs
- **NBA/Basketball**: Identify fouls based on collision and contact zones

-- -- --


## ✅ Step 1: Introduction to VAR/Umpire Automation

**Goal**: Understand how AI can support officiating decisions.

###Topics:
- What is VAR in modern sports?
- How AI & CV can enhance referee accuracy
- Overview of event output format (table with timestamp, action, player, clip)

Screenshot 2025-05-19 085802.png

##✅ Step 2: Install and Load Libraries

**Goal**: Prepare the environment.

**Topics**:
- Use OpenCV for video.
- Optionally integrate `mediapipe`

`!pip install opencv-python moviepy`

------

In [1]:
!pip install opencv-python moviepy
!pip install yt-dlp
!apt-get install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


##✅ Step 3: Load YouTube Video & Extract Frames

**Goal**: Load a sports video and extract frames/timestamps

###Topics:
- Download video using pytube
- Extract video frames and frame rate for time conversion

In [2]:
import yt_dlp
import cv2

url = "https://www.youtube.com/watch?v=YOUR_VIDEO"
url = "https://www.youtube.com/watch?v=LfG4TxYpaVk&t"
ydl_opts = {
    'format': 'bestvideo[ext=mp4][vcodec^=avc1]',  # video-only, mp4 container, AVC codec
    'outtmpl': 'match.mp4',
    'postprocessors': []  # No merging
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

cap = cv2.VideoCapture("match.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
print("Video FPS:", fps)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LfG4TxYpaVk&t
[youtube] LfG4TxYpaVk: Downloading webpage
[youtube] LfG4TxYpaVk: Downloading tv client config
[youtube] LfG4TxYpaVk: Downloading player a10d7fcc-main
[youtube] LfG4TxYpaVk: Downloading tv player API JSON
[youtube] LfG4TxYpaVk: Downloading ios player API JSON
[youtube] LfG4TxYpaVk: Downloading m3u8 information
[info] LfG4TxYpaVk: Downloading 1 format(s): 299
[download] Destination: match.mp4
[download] 100% of  582.65MiB in 00:00:16 at 36.16MiB/s  
Video FPS: 60.0


##✅ Step 4: Pose/Action/Event Detection with OpenCV

**Goal**: Spot actions like collision, offside, or contact.

###Topics:
- Use basic object tracking (color or motion)
- (Optional) Integrate pose models like MediaPipe, YOLO, or OpenPose for player detection

In [3]:
# Simple motion detection example
prev_frame = None
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    if prev_frame is None:
        prev_frame = gray
        continue

    frame_delta = cv2.absdiff(prev_frame, gray)
    thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)[1]
    motion = cv2.countNonZero(thresh)

    if motion > 50000:  # example threshold
        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
        print("High activity at", timestamp, "sec")

    prev_frame = gray

High activity at 1.7166666666666666 sec
High activity at 6.733333333333333 sec
High activity at 6.75 sec
High activity at 6.766666666666667 sec
High activity at 6.783333333333333 sec
High activity at 6.8 sec
High activity at 6.816666666666666 sec
High activity at 6.833333333333333 sec


KeyboardInterrupt: 

##✅ Step 5: Label Detected Event (Example: LBW in Cricket)

**Goal**: Create event logic and tag output

###Topics:
- Manually or algorithmically tag events
- Store in structured format

In [ ]:
# Example event list
events = [
    {"timestamp": "2 min 30 sec", "sports_type": "Boxing", "extracted_data": "Left upper", "player": "Manny Pacio", "clip": "Clip URL"},
    {"timestamp": "4 min 18 sec", "sports_type": "Boxing", "extracted_data": "Rear hook", "player": "Floyd Mayweather", "clip": "Clip URL"}
]

## ✅ Step 6: Generate Event Clips with MoviePy

**Goal**: Slice the original video into short event clips

###Topics:
- Convert seconds to clip
- Save as files or upload links

In [ ]:
from moviepy.editor import VideoFileClip

video = VideoFileClip("match.mp4")
highlight = video.subclip(150, 160)  # 2 min 30 sec to 2 min 40 sec
highlight.write_videofile("event1.mp4")

##✅ Step 7: Output Final Event Table (Like Your Screenshot)

**Goal**: Tabulate and export event data

###Topics:
- Format using Pandas or markdown
- Export as CSV or embed as HTML table

In [ ]:
import pandas as pd

df = pd.DataFrame(events)
df.to_csv("var_events.csv", index=False)

## 🎯 Hackathon Task Ideas for VAR Track

- ⚽ Soccer Offside Detector: Use YOLO to detect player and ball positions + relative locations
-🏏 LBW Predictor in Cricket: Use pose & ball trajectory to flag possible LBWs

- 🏀 Basketball Foul Classifier: Use collision/motion zone modeling to detect fouls